In [432]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno  
import datatable as dt
import re
from sklearn import metrics

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [433]:
train = pd.read_csv('train.csv')
train.head(1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [434]:
train.drop(['Id','PoolQC','Alley','Street','Utilities',
            'Condition2','MasVnrArea','Fence','MiscFeature',
            'BsmtHalfBath','EnclosedPorch','FireplaceQu',
           '3SsnPorch','ScreenPorch'],axis=1,inplace=True)

In [435]:
train.head(1)

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Reg,Lvl,Inside,Gtl,CollgCr,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,2,2008,WD,Normal,208500


In [436]:
check_nan = train.isna().sum().sort_values(ascending=False).to_frame(name='Count')
check_nan['%'] = check_nan.Count / len(train) * 100
check_nan.head(13)

,Count,%
LotFrontage,259,17.739726
GarageType,81,5.547945
GarageCond,81,5.547945
GarageYrBlt,81,5.547945
GarageFinish,81,5.547945
GarageQual,81,5.547945
BsmtFinType2,38,2.602740
BsmtExposure,38,2.602740
BsmtFinType1,37,2.534247
BsmtCond,37,2.534247


#### Filling NaN

In [437]:
train.LotFrontage.fillna(train.LotFrontage.median(),inplace=True)
train.LotFrontage = train.LotFrontage.astype('int')

train.GarageYrBlt.fillna(2005,inplace=True)
train.GarageYrBlt = train.GarageYrBlt.astype('int')

train.GarageCond.replace(np.NaN,'TA',inplace=True)
train.GarageFinish.replace(np.NaN,'Unf',inplace=True)
train.GarageQual.replace(np.NaN,'TA',inplace=True)
train.GarageType.replace(np.NaN,'Attchd',inplace=True)
train.BsmtFinType2.replace(np.NaN,'Unf',inplace=True)
train.BsmtExposure.replace(np.NaN,'No',inplace=True)
train.BsmtFinType1.replace(np.NaN,'Unf',inplace=True)
train.BsmtCond.replace(np.NaN,'TA',inplace=True)
train.BsmtQual.replace(np.NaN,'TA',inplace=True)
train.MasVnrType.replace(np.NaN,'None',inplace=True)
train.Electrical.replace(np.NaN,'SBrkr',inplace=True)


#### Encoding categorial data

In [438]:
columns_to_encode = ['MSZoning','LotShape','LandContour',
                     'LandSlope','LotConfig','BsmtFinType2',
                     'Neighborhood','Condition1','Heating',
                     'BldgType','HouseStyle','RoofStyle',
                     'RoofMatl','Exterior1st','Exterior2nd',
                     'MasVnrType','ExterQual','ExterCond',
                     'Foundation','BsmtQual','BsmtCond',
                     'BsmtExposure','BsmtFinType1',
                     'HeatingQC','CentralAir','Electrical',
                     'KitchenQual','Functional','GarageType',
                     'GarageFinish','GarageQual','GarageCond',
                     'PavedDrive','SaleType','SaleCondition']
for column in columns_to_encode:
    encoder = LabelEncoder()
    train[column]= encoder.fit_transform(train[column].values)

In [439]:
train.head(1)

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,3,65,8450,3,3,4,0,5,2,0,5,7,5,2003,2003,1,1,12,13,1,2,4,2,2,3,3,2,706,5,0,150,856,1,0,1,4,856,854,0,1710,1,2,1,3,1,2,8,6,0,1,2003,1,2,548,4,4,2,0,61,0,0,2,2008,8,4,208500


#### Discretization

In [440]:
columns_to_des = ['LotFrontage','LotArea','BsmtFinSF1',
                  'BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',
                  '1stFlrSF','2ndFlrSF','GrLivArea','GarageArea',
                  'WoodDeckSF','OpenPorchSF']
list_number = []
for i in columns_to_des:
    # print(train[i].describe())
    number = int(train[i].max()//train[i].std())
    if number > 10:
        number = 10
    if i == "BsmtFinSF2" or i == "2ndFlrSF" or i == "WoodDeckSF":
        number = 1
    if i == "BsmtFinSF1":
        train["BsmtFinSF1"] = pd.qcut(train["BsmtFinSF1"], q=3,labels=[i for i in range(3)])
        list_number.append(3)
        continue
    if i == "OpenPorchSF":
        train["OpenPorchSF"] = pd.qcut(train["OpenPorchSF"], q=2,labels=[i for i in range(2)])
        list_number.append(2)
        continue
    list_number.append(number)
    train[i] = pd.qcut(train[i], q=number,labels=[i for i in range(number)])
# train["OpenPorchSF"].describe()
# number = 2
# train["OpenPorchSF"] = pd.qcut(train["OpenPorchSF"], q=number,labels=[i for i in range(number)])


In [441]:
list_number

[10, 10, 3, 1, 5, 10, 10, 1, 10, 6, 1, 2]

##### train["BsmtFinSF2"].head()

#### Test data

In [442]:
test = pd.read_csv('test.csv')
test.head(1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal


In [443]:
test.drop(['Id','PoolQC','Alley','Street','Utilities',
            'Condition2','MasVnrArea','Fence','MiscFeature',
            'BsmtHalfBath','EnclosedPorch','FireplaceQu',
           '3SsnPorch','ScreenPorch'],axis=1,inplace=True)

In [444]:
check_nan = test.isna().sum().sort_values(ascending=False).to_frame(name='Count')
check_nan['%'] = check_nan.Count / len(test) * 100
check_nan.head(25)

,Count,%
LotFrontage,227,15.558602
GarageFinish,78,5.346127
GarageCond,78,5.346127
GarageQual,78,5.346127
GarageYrBlt,78,5.346127
GarageType,76,5.209047
BsmtCond,45,3.084304
BsmtExposure,44,3.015764
BsmtQual,44,3.015764
BsmtFinType2,42,2.878684


#### Filling NaN test

In [445]:
test.LotFrontage.fillna(test.LotFrontage.median(),inplace=True)
test.LotFrontage = test.LotFrontage.astype('int')

test.GarageFinish.replace(np.NaN,'Unf',inplace=True)
test.GarageCond.replace(np.NaN,'TA',inplace=True)
test.GarageQual.replace(np.NaN,'TA',inplace=True)
test.GarageType.replace(np.NaN,'Attchd',inplace=True)
test.BsmtCond.replace(np.NaN,'TA',inplace=True)
test.BsmtExposure.replace(np.NaN,'No',inplace=True)
test.BsmtQual.replace(np.NaN,'TA',inplace=True)
test.BsmtFinType2.replace(np.NaN,'Unf',inplace=True)
test.BsmtFinType1.replace(np.NaN,'GLQ',inplace=True)
test.MasVnrType.replace(np.NaN,'None',inplace=True)
test.MSZoning.replace(np.NaN,'RL',inplace=True)
test.Functional.replace(np.NaN,'Typ',inplace=True)
test.Exterior1st.replace(np.NaN,'VinylSd',inplace=True)
test.Exterior2nd.replace(np.NaN,'VinylSd',inplace=True)
test.KitchenQual.replace(np.NaN,'TA',inplace=True)
test.SaleType.replace(np.NaN,'WD',inplace=True)


test.BsmtFullBath.fillna(0,inplace=True)
test.BsmtFullBath = test.BsmtFullBath.astype('int')

test.GarageCars.fillna(2,inplace=True)
test.GarageCars = test.GarageCars.astype('int')

test.BsmtFinSF2.fillna(test.BsmtFinSF2.median(),inplace=True)
test.BsmtFinSF2 = test.LotFrontage.astype('int')

test.TotalBsmtSF.fillna(test.TotalBsmtSF.median(),inplace=True)
test.TotalBsmtSF = test.LotFrontage.astype('int')

test.BsmtUnfSF.fillna(test.BsmtUnfSF.median(),inplace=True)
test.BsmtUnfSF = test.LotFrontage.astype('int')

test.GarageCars.fillna(test.GarageCars.median(),inplace=True)
test.GarageCars = test.LotFrontage.astype('int')

test.BsmtFinSF1.fillna(test.BsmtFinSF1.median(),inplace=True)
test.BsmtFinSF1 = test.LotFrontage.astype('int')

test.GarageArea.fillna(test.GarageArea.median(),inplace=True)
test.GarageArea = test.LotFrontage.astype('int')

test.GarageYrBlt.fillna(2005,inplace=True)
test.GarageYrBlt = train.GarageYrBlt.astype('int')

#### Encoding test data

In [446]:
for column in columns_to_encode:
    encoder = LabelEncoder()
    test[column]= encoder.fit_transform(test[column].values)

In [447]:
columns_to_des = ['LotFrontage','LotArea','BsmtFinSF1',
                  'BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',
                  '1stFlrSF','2ndFlrSF','GrLivArea','GarageArea',
                  'WoodDeckSF','OpenPorchSF']
num=0
for i in columns_to_des:
    test[i] = pd.qcut(test[i], q=list_number[num],labels=[i for i in range(list_number[num])])
    num+=1
# test["LotArea"] = pd.qcut(test["LotArea"], q=10,labels=[i for i in range(10)])

#### Checking final data

In [448]:
test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,2,7,7,3,3,4,0,12,1,0,2,5,6,1961,1961,1,0,10,12,2,3,4,1,3,3,3,4,2,3,0,3,7,0,4,1,3,2,0,0,0,0,1,0,2,1,3,5,6,0,1,2003,2,80,4,3,4,2,0,0,0,0,6,2010,8,4
1,20,3,8,9,0,3,0,0,12,2,0,2,6,6,1958,1958,3,0,11,13,1,3,4,1,3,3,3,0,2,5,0,4,8,0,4,1,3,7,0,0,4,0,1,1,3,1,2,6,6,0,1,1976,2,81,4,3,4,2,0,1,0,12500,6,2010,8,4
2,60,3,6,8,0,3,4,0,8,2,0,4,5,5,1997,1998,1,0,10,12,2,3,4,2,2,3,3,2,2,5,0,3,6,0,2,1,3,3,0,0,6,0,2,1,3,1,3,6,6,1,1,2001,0,74,4,3,4,2,0,1,0,0,3,2010,8,4
3,60,3,7,5,0,3,4,0,8,2,0,4,6,6,1998,1998,1,0,10,12,1,3,4,2,3,3,3,2,2,5,0,3,7,0,0,1,3,3,0,0,6,0,2,1,3,1,2,7,6,1,1,1998,0,78,4,3,4,2,0,1,0,0,6,2010,8,4
4,120,3,0,1,0,1,4,0,22,2,4,2,8,5,1992,1992,1,0,6,6,2,2,4,2,2,3,3,0,0,5,0,0,0,0,0,1,3,6,0,0,3,0,2,0,2,1,2,5,6,0,1,2000,1,43,0,3,4,2,0,1,0,0,1,2010,8,4


In [449]:
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,3,3,3,3,3,4,0,5,2,0,5,7,5,2003,2003,1,1,12,13,1,2,4,2,2,3,3,2,2,5,0,0,3,1,0,1,4,2,0,0,7,1,2,1,3,1,2,8,6,0,1,2003,1,2,4,4,4,2,0,1,0,0,2,2008,8,4,208500
1,20,3,7,5,3,3,2,0,24,1,0,2,6,8,1976,1976,1,1,8,8,2,3,4,1,2,3,1,0,2,5,0,1,7,1,0,1,4,6,0,0,3,0,2,0,3,1,3,6,6,1,1,1976,1,2,2,4,4,2,0,0,0,0,5,2007,8,4,181500
2,60,3,3,7,0,3,4,0,5,2,0,5,7,5,2001,2002,1,1,12,13,1,2,4,2,2,3,2,2,1,5,0,2,4,1,0,1,4,3,0,0,7,1,2,1,3,1,2,6,6,1,1,2001,1,2,4,4,4,2,0,1,0,0,9,2008,8,4,223500
3,70,3,2,5,0,3,0,0,6,2,0,5,7,5,1915,1970,1,1,13,15,2,3,4,0,3,1,3,0,1,5,0,2,2,1,2,1,4,3,0,0,7,1,1,0,3,1,2,7,6,1,5,1998,2,3,4,4,4,2,0,1,0,0,2,2006,8,0,140000
4,60,3,8,8,0,3,2,0,15,2,0,5,8,5,2000,2000,1,1,12,13,1,2,4,2,2,3,0,2,2,5,0,2,6,1,0,1,4,5,0,0,9,1,2,1,4,1,2,9,6,1,1,2000,1,3,5,4,4,2,0,1,0,0,12,2008,8,4,250000


#### Scaling 

In [450]:
# from sklearn.preprocessing import StandardScaler
# # scaler = MinMaxScaler()
# scaler = StandardScaler()
# test_n = scaler.fit_transform(test)
# test = pd.DataFrame(test_n, columns = test.columns)

# train_n = scaler.fit_transform(train)
# train = pd.DataFrame(train_n, columns = train.columns)




In [451]:
train.describe()

,MSSubClass,MSZoning,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,LowQualFinSF,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageQual,GarageCond,PavedDrive,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,3.028767,1.942466,2.777397,3.019178,0.062329,12.251370,2.031507,0.493151,3.038356,6.099315,5.575342,1971.267808,1984.865753,1.410274,1.075342,9.624658,10.339726,1.762329,2.539726,3.733562,1.396575,2.280137,2.817123,2.284247,2.789726,4.715753,1.036301,1.538356,0.934932,3.682192,5.844521,0.425342,1.565068,0.382877,2.866438,1.046575,2.339726,6.517808,5.749315,0.613014,2.208219,1979.976027,1.228767,1.767123,3.871918,3.905479,1.856164,2.758904,43.489041,6.321918,2007.815753,7.513014,3.770548,180921.195890
std,42.300571,0.632017,1.409156,0.707666,1.622634,0.276232,6.013735,0.868515,1.198277,1.911305,1.382997,1.112799,30.202904,20.645407,0.834998,0.599127,3.197659,3.540570,0.614274,0.693995,0.731807,0.722394,0.864510,0.650919,1.138472,1.837550,0.925252,0.295124,1.739524,0.246731,1.051301,48.623081,0.518911,0.550916,0.502885,0.815778,0.220338,0.830161,1.625393,0.979659,0.644666,1.759864,24.749688,0.811835,0.747315,0.594151,0.508297,0.496592,40.177307,496.123024,2.703626,1.328095,1.552100,1.100854,79442.502883
min,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,0.000000,0.000000,34900.000000
25%,20.000000,3.000000,0.000000,3.000000,2.000000,0.000000,7.000000,2.000000,0.000000,2.000000,5.000000,5.000000,1954.000000,1967.000000,1.000000,1.000000,8.000000,8.000000,1.000000,2.000000,4.000000,1.000000,2.000000,3.000000,2.000000,1.000000,5.000000,1.000000,0.000000,1.000000,4.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,2.000000,5.000000,6.000000,0.000000,1.000000,1962.000000,1.000000,1.000000,4.000000,4.000000,2.000000,0.000000,0.000000,5.000000,2007.000000,8.000000,4.000000,129975.000000
50%,50.000000,3.000000,3.000000,3.000000,4.000000,0.000000,12.000000,2.000000,0.000000,2.000000,6.000000,5.000000,1973.000000,1994.000000,1.000000,1.000000,12.000000,13.000000,2.000000,3.000000,4.000000,1.000000,2.000000,3.000000,3.000000,2.000000,5.000000,1.000000,0.000000,1.000000,4.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,3.000000,6.000000,6.000000,1.000000,1.000000,1984.500000,1.000000,2.000000,4.000000,4.000000,2.000000,0.000000,0.000000,6.000000,2008.000000,8.000000,4.000000,163000.000000
75%,70.000000,3.000000,3.000000,3.000000,4.000000,0.000000,17.000000,2.000000,0.000000,5.000000,7.000000,6.000000,2000.000000,2004.000000,1.000000,1.0000

In [452]:
X,y = train.drop('SalePrice',axis=1),train.SalePrice

In [453]:
model = LinearRegression().fit(X, y)
pred = model.predict(test)

In [454]:
print(pred)

[ 698767.79841534  742445.19034626  710211.19701488 ... 1332757.81409476
  568768.82200048  775858.12879719]


In [455]:
test_t = pd.read_csv('test.csv')
sub = pd.DataFrame({'Id':test_t.Id,'SalePrice':pred})
sub.to_csv('submission_house.csv',index=False)
sub.head(5)

,Id,SalePrice
0,1461,698767.798415
1,1462,742445.190346
2,1463,710211.197015
3,1464,760092.144834
4,1465,472158.123340
